In [11]:
import pandas as pd
import numpy as np
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import HiveContext

from pyspark.sql import SparkSession

In [12]:
sample_submission = "./sample_submission.csv"
departments = "./departments.csv"
order_products__prior = "./order_products__prior.csv"
order_products__train = "./order_products__train.csv"
orders = "./orders.csv"
products = "./products.csv"
aisles = "./aisles.csv"

In [13]:
sample_submission_df = pd.read_csv(sample_submission)

In [14]:
departments_df = pd.read_csv(departments)

In [15]:
order_products__prior_df = pd.read_csv(order_products__prior)

In [16]:
order_products__train_df = pd.read_csv(order_products__train)

In [17]:
orders_df = pd.read_csv(orders)

In [18]:
products_df = pd.read_csv(products)

In [19]:
aisles_df = pd.read_csv(aisles)

In [26]:
sample_submission_cols = sample_submission_df.columns
sample_submission_cols

Index(['order_id', 'products'], dtype='object')

In [27]:
departments_cols = departments_df.columns
departments_cols

Index(['department_id', 'department'], dtype='object')

In [28]:
order_products__prior_cols = order_products__prior_df.columns
order_products__prior_cols

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')

In [29]:
order_products__train_cols = order_products__train_df.columns
order_products__train_cols

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')

In [30]:
orders_cols = orders_df.columns
orders_cols

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')

In [31]:
products_cols = products_df.columns
products_cols

Index(['product_id', 'product_name', 'aisle_id', 'department_id'], dtype='object')

In [32]:
aisles_cols = aisles_df.columns
aisles_cols

Index(['aisle_id', 'aisle'], dtype='object')

In [33]:
aisles_df[aisles_cols].count()

aisle_id    134
aisle       134
dtype: int64

In [34]:
departments_df[departments_cols].count()

department_id    21
department       21
dtype: int64

In [35]:
products_df[products_cols].count()

product_id       49688
product_name     49688
aisle_id         49688
department_id    49688
dtype: int64

In [36]:
order_products__prior_df[order_products__prior_cols].count()

order_id             32434489
product_id           32434489
add_to_cart_order    32434489
reordered            32434489
dtype: int64

In [37]:
order_products__train_df[order_products__train_cols].count()

order_id             1384617
product_id           1384617
add_to_cart_order    1384617
reordered            1384617
dtype: int64

In [38]:
orders_df[orders_cols].count()

order_id                  3421083
user_id                   3421083
eval_set                  3421083
order_number              3421083
order_dow                 3421083
order_hour_of_day         3421083
days_since_prior_order    3214874
dtype: int64

In [39]:
sample_submission_df[sample_submission_cols].count()

order_id    75000
products    75000
dtype: int64

In [41]:
# aisles_df[aisles_cols].describe()

In [43]:
# departments_df[departments_cols].describe()

In [44]:
products_df[products_cols].describe()

,product_id,aisle_id,department_id
count,49688.000000,49688.000000,49688.000000
mean,24844.500000,67.769582,11.728687
std,14343.834425,38.316162,5.850410
min,1.000000,1.000000,1.000000
25%,12422.750000,35.000000,7.000000
50%,24844.500000,69.000000,13.000000
75%,37266.250000,100.000000,17.000000
max,49688.000000,134.000000,21.000000


In [45]:
order_products__prior_df[order_products__prior_cols].describe()

,order_id,product_id,add_to_cart_order,reordered
count,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07
mean,1.710749e+06,2.557634e+04,8.351076e+00,5.896975e-01
std,9.873007e+05,1.409669e+04,7.126671e+00,4.918886e-01
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.559430e+05,1.353000e+04,3.000000e+00,0.000000e+00
50%,1.711048e+06,2.525600e+04,6.000000e+00,1.000000e+00
75%,2.565514e+06,3.793500e+04,1.100000e+01,1.000000e+00
max,3.421083e+06,4.968800e+04,1.450000e+02,1.000000e+00


In [46]:
order_products__train_df[order_products__train_cols].describe()

,order_id,product_id,add_to_cart_order,reordered
count,1.384617e+06,1.384617e+06,1.384617e+06,1.384617e+06
mean,1.706298e+06,2.555624e+04,8.758044e+00,5.985944e-01
std,9.897326e+05,1.412127e+04,7.423936e+00,4.901829e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.433700e+05,1.338000e+04,3.000000e+00,0.000000e+00
50%,1.701880e+06,2.529800e+04,7.000000e+00,1.000000e+00
75%,2.568023e+06,3.794000e+04,1.200000e+01,1.000000e+00
max,3.421070e+06,4.968800e+04,8.000000e+01,1.000000e+00


In [47]:
orders_df[orders_cols].describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


In [48]:
sample_submission_df[sample_submission_cols].describe()

,order_id
count,7.500000e+04
mean,1.709686e+06
std,9.878542e+05
min,1.700000e+01
25%,8.557232e+05
50%,1.710152e+06
75%,2.562794e+06
max,3.421054e+06


In [ ]:
# sample_submission_df['order_id']
# sample_submission_df['products']

# departments_df['department_id']
# departments_df['department']

# order_products__prior_df['order_id']
# order_products__prior_df['product_id']
# order_products__prior_df['add_to_cart_order']
# order_products__prior_df['reordered']

# order_products__train_df['order_id']
# order_products__train_df['product_id']
# order_products__train_df['add_to_cart_order']
# order_products__train_df['reordered']

# orders_df['order_id']
# orders_df['user_id']
# orders_df['eval_set']
# orders_df['order_number']
# orders_df['order_dow']
# orders_df['order_hour_of_day']
# orders_df['days_since_prior_order']

# products_df['product_id']
# products_df['product_name']
# products_df['aisle_id']
# products_df['department_id']

# aisles_df['aisle_id']
# aisles_df['aisle']

In [54]:
pd.qcut(sample_submission_df['order_id'], 4, labels=False)

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
74970    3
74971    3
74972    3
74973    3
74974    3
74975    3
74976    3
74977    3
74978    3
74979    3
74980    3
74981    3
74982    3
74983    3
74984    3
74985    3
74986    3
74987    3
74988    3
74989    3
74990    3
74991    3
74992    3
74993    3
74994    3
74995    3
74996    3
74997    3
74998    3
74999    3
Name: order_id, Length: 75000, dtype: int64

In [56]:
pd.qcut(order_products__prior_df['order_id'], 4, labels=False)

0           0
1           0
2           0
3           0
4           0
5           0
6           0
7           0
8           0
9           0
10          0
11          0
12          0
13          0
14          0
15          0
16          0
17          0
18          0
19          0
20          0
21          0
22          0
23          0
24          0
25          0
26          0
27          0
28          0
29          0
           ..
32434459    3
32434460    3
32434461    3
32434462    3
32434463    3
32434464    3
32434465    3
32434466    3
32434467    3
32434468    3
32434469    3
32434470    3
32434471    3
32434472    3
32434473    3
32434474    3
32434475    3
32434476    3
32434477    3
32434478    3
32434479    3
32434480    3
32434481    3
32434482    3
32434483    3
32434484    3
32434485    3
32434486    3
32434487    3
32434488    3
Name: order_id, Length: 32434489, dtype: int64

In [57]:
pd.qcut(order_products__prior_df['product_id'], 4, labels=False)

0           2
1           2
2           0
3           3
4           2
5           1
6           3
7           0
8           3
9           2
10          1
11          1
12          1
13          1
14          3
15          1
16          2
17          3
18          2
19          3
20          2
21          0
22          1
23          1
24          2
25          3
26          1
27          1
28          2
29          3
           ..
32434459    3
32434460    2
32434461    0
32434462    1
32434463    0
32434464    3
32434465    3
32434466    0
32434467    2
32434468    0
32434469    1
32434470    2
32434471    0
32434472    1
32434473    0
32434474    1
32434475    3
32434476    2
32434477    0
32434478    3
32434479    0
32434480    3
32434481    1
32434482    1
32434483    2
32434484    3
32434485    0
32434486    0
32434487    1
32434488    0
Name: product_id, Length: 32434489, dtype: int64

In [58]:
pd.qcut(order_products__prior_df['add_to_cart_order'], 4, labels=False)

0           0
1           0
2           0
3           1
4           1
5           1
6           2
7           2
8           2
9           0
10          0
11          0
12          1
13          1
14          1
15          2
16          2
17          0
18          0
19          0
20          1
21          1
22          1
23          2
24          2
25          2
26          2
27          2
28          3
29          3
           ..
32434459    1
32434460    1
32434461    1
32434462    2
32434463    2
32434464    2
32434465    0
32434466    0
32434467    0
32434468    1
32434469    1
32434470    1
32434471    2
32434472    0
32434473    0
32434474    0
32434475    1
32434476    1
32434477    1
32434478    2
32434479    0
32434480    0
32434481    0
32434482    1
32434483    1
32434484    1
32434485    2
32434486    2
32434487    2
32434488    2
Name: add_to_cart_order, Length: 32434489, dtype: int64

In [60]:

pd.qcut(order_products__train_df['order_id'], 4, labels=False)

0          0
1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
10         0
11         0
12         0
13         0
14         0
15         0
16         0
17         0
18         0
19         0
20         0
21         0
22         0
23         0
24         0
25         0
26         0
27         0
28         0
29         0
          ..
1384587    3
1384588    3
1384589    3
1384590    3
1384591    3
1384592    3
1384593    3
1384594    3
1384595    3
1384596    3
1384597    3
1384598    3
1384599    3
1384600    3
1384601    3
1384602    3
1384603    3
1384604    3
1384605    3
1384606    3
1384607    3
1384608    3
1384609    3
1384610    3
1384611    3
1384612    3
1384613    3
1384614    3
1384615    3
1384616    3
Name: order_id, Length: 1384617, dtype: int64

In [61]:
pd.qcut(order_products__train_df['product_id'], 4, labels=False)

0          3
1          0
2          0
3          3
4          3
5          0
6          3
7          1
8          3
9          1
10         3
11         3
12         3
13         2
14         3
15         3
16         0
17         1
18         0
19         1
20         1
21         2
22         2
23         3
24         3
25         1
26         2
27         3
28         2
29         2
          ..
1384587    2
1384588    0
1384589    0
1384590    0
1384591    3
1384592    1
1384593    2
1384594    1
1384595    2
1384596    2
1384597    0
1384598    0
1384599    1
1384600    1
1384601    0
1384602    1
1384603    0
1384604    2
1384605    0
1384606    0
1384607    2
1384608    2
1384609    2
1384610    3
1384611    1
1384612    1
1384613    2
1384614    2
1384615    1
1384616    0
Name: product_id, Length: 1384617, dtype: int64

In [62]:
pd.qcut(order_products__train_df['add_to_cart_order'], 4, labels=False)

0          0
1          0
2          0
3          1
4          1
5          1
6          1
7          2
8          0
9          0
10         0
11         1
12         1
13         1
14         1
15         2
16         0
17         0
18         0
19         1
20         1
21         1
22         1
23         2
24         2
25         0
26         0
27         0
28         1
29         1
          ..
1384587    0
1384588    1
1384589    1
1384590    1
1384591    0
1384592    0
1384593    0
1384594    1
1384595    1
1384596    1
1384597    0
1384598    0
1384599    0
1384600    1
1384601    1
1384602    0
1384603    0
1384604    0
1384605    1
1384606    1
1384607    1
1384608    1
1384609    2
1384610    0
1384611    0
1384612    0
1384613    1
1384614    0
1384615    0
1384616    0
Name: add_to_cart_order, Length: 1384617, dtype: int64

In [63]:
pd.qcut(order_products__train_df['reordered'], 4, labels=False)

ValueError: Bin edges must be unique: array([0, 0, 1, 1, 1]).
You can drop duplicate edges by setting the 'duplicates' kwarg

In [64]:

pd.qcut(orders_df['order_id'], 4, labels=False)

0          2
1          2
2          0
3          2
4          0
5          3
6          0
7          3
8          2
9          2
10         1
11         2
12         1
13         2
14         0
15         1
16         1
17         3
18         0
19         2
20         1
21         1
22         3
23         3
24         0
25         1
26         1
27         0
28         3
29         2
          ..
3421053    3
3421054    0
3421055    1
3421056    3
3421057    3
3421058    3
3421059    0
3421060    3
3421061    0
3421062    2
3421063    3
3421064    3
3421065    2
3421066    1
3421067    2
3421068    0
3421069    3
3421070    2
3421071    1
3421072    0
3421073    2
3421074    3
3421075    0
3421076    2
3421077    2
3421078    2
3421079    2
3421080    0
3421081    3
3421082    0
Name: order_id, Length: 3421083, dtype: int64

In [67]:
pd.qcut(orders_df['order_number'], 4, labels=False)

0          0
1          0
2          0
3          0
4          0
5          1
6          1
7          1
8          1
9          1
10         1
11         0
12         0
13         0
14         0
15         0
16         1
17         1
18         1
19         1
20         1
21         1
22         2
23         2
24         2
25         2
26         0
27         0
28         0
29         0
          ..
3421053    3
3421054    3
3421055    3
3421056    3
3421057    3
3421058    3
3421059    3
3421060    3
3421061    3
3421062    3
3421063    3
3421064    3
3421065    3
3421066    3
3421067    3
3421068    3
3421069    0
3421070    0
3421071    0
3421072    0
3421073    0
3421074    1
3421075    1
3421076    1
3421077    1
3421078    1
3421079    1
3421080    2
3421081    2
3421082    2
Name: order_number, Length: 3421083, dtype: int64

In [68]:
pd.qcut(orders_df['order_dow'], 4, labels=False)

0          1
1          1
2          1
3          2
4          2
5          1
6          0
7          0
8          0
9          2
10         2
11         1
12         2
13         0
14         1
15         1
16         1
17         1
18         0
19         1
20         0
21         0
22         0
23         2
24         1
25         0
26         0
27         1
28         1
29         1
          ..
3421053    1
3421054    3
3421055    1
3421056    0
3421057    1
3421058    2
3421059    1
3421060    0
3421061    2
3421062    3
3421063    1
3421064    0
3421065    1
3421066    0
3421067    0
3421068    2
3421069    1
3421070    1
3421071    2
3421072    0
3421073    2
3421074    0
3421075    1
3421076    1
3421077    2
3421078    2
3421079    2
3421080    0
3421081    0
3421082    3
Name: order_dow, Length: 3421083, dtype: int64

In [69]:
pd.qcut(orders_df['order_hour_of_day'], 4, labels=False)

0          0
1          0
2          1
3          0
4          2
5          0
6          0
7          2
8          2
9          0
10         0
11         1
12         0
13         0
14         0
15         1
16         0
17         1
18         2
19         0
20         1
21         0
22         0
23         1
24         0
25         1
26         2
27         3
28         2
29         3
          ..
3421053    3
3421054    2
3421055    1
3421056    2
3421057    2
3421058    0
3421059    2
3421060    3
3421061    2
3421062    2
3421063    1
3421064    0
3421065    0
3421066    1
3421067    3
3421068    1
3421069    1
3421070    3
3421071    1
3421072    0
3421073    2
3421074    2
3421075    1
3421076    3
3421077    2
3421078    3
3421079    0
3421080    1
3421081    1
3421082    2
Name: order_hour_of_day, Length: 3421083, dtype: int64

In [70]:
pd.qcut(orders_df['days_since_prior_order'], 4, labels=False)

0          NaN
1          2.0
2          3.0
3          3.0
4          3.0
5          3.0
6          3.0
7          2.0
8          0.0
9          3.0
10         2.0
11         NaN
12         2.0
13         0.0
14         2.0
15         2.0
16         2.0
17         2.0
18         3.0
19         2.0
20         1.0
21         3.0
22         3.0
23         3.0
24         2.0
25         3.0
26         NaN
27         2.0
28         3.0
29         3.0
          ... 
3421053    0.0
3421054    2.0
3421055    0.0
3421056    3.0
3421057    2.0
3421058    2.0
3421059    2.0
3421060    1.0
3421061    0.0
3421062    0.0
3421063    0.0
3421064    2.0
3421065    2.0
3421066    1.0
3421067    1.0
3421068    0.0
3421069    NaN
3421070    1.0
3421071    3.0
3421072    3.0
3421073    0.0
3421074    0.0
3421075    2.0
3421076    3.0
3421077    3.0
3421078    3.0
3421079    3.0
3421080    3.0
3421081    1.0
3421082    3.0
Name: days_since_prior_order, Length: 3421083, dtype: float64

In [71]:
pd.qcut(products_df['product_id'], 4, labels=False)

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
49658    3
49659    3
49660    3
49661    3
49662    3
49663    3
49664    3
49665    3
49666    3
49667    3
49668    3
49669    3
49670    3
49671    3
49672    3
49673    3
49674    3
49675    3
49676    3
49677    3
49678    3
49679    3
49680    3
49681    3
49682    3
49683    3
49684    3
49685    3
49686    3
49687    3
Name: product_id, Length: 49688, dtype: int64

In [73]:
pd.qcut(products_df['aisle_id'], 4, labels=False)

0        1
1        3
2        2
3        1
4        0
5        0
6        2
7        3
8        3
9        3
10       0
11       3
12       0
13       2
14       1
15       3
16       0
17       2
18       1
19       2
20       1
21       0
22       1
23       1
24       0
25       1
26       3
27       3
28       2
29       1
        ..
49658    0
49659    0
49660    3
49661    3
49662    1
49663    2
49664    1
49665    1
49666    1
49667    3
49668    2
49669    3
49670    1
49671    0
49672    3
49673    1
49674    0
49675    1
49676    1
49677    3
49678    1
49679    2
49680    1
49681    2
49682    2
49683    3
49684    1
49685    3
49686    1
49687    2
Name: aisle_id, Length: 49688, dtype: int64

In [ ]:
pd.qcut(products_df['department_id'], 4, labels=False)

pd.qcut(aisles_df['aisle'], 4, labels=False)

pd.qcut(departments_df['department'], 4, labels=False)